In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("MRI.ipynb")

# EE 120 Lab 5: Magnetic Resonance Imaging

**Signals and Systems** at UC Berkeley

Acknowledgements:
* **Spring 2023** (v1.0): Gavin Zhang

# Dependencies
Run the cell below to import the necessary modules.


In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pylab as plt


# Background
Magnetic Resonance Imaging (MRI) is a medical imaging technique used to visualize the internal structure of the body. The fundamentals of MRI rely on physical properties of magnetism and the intracies of the Fourier transform. In fact, EE225E (the MRI class) only requires EE120 as a prerequisite.

Let's go through a high-level overview of how MRI works. An MRI machine has very powerful magnetic coils, and when a patient is placed inside an MRI machine, the machine generates a powerful, uniform magnetic field $B_0$ around their body.  
[<img src="images/scanner.PNG" width="500"/>](images/scanner.PNG)

Protons inside the human body have their own magnetic moment and spins. Normally, the protons in the body are all spinning and pointing in different directions, so their magnetic moments cancel each other out. However, when a powerful magnetic field is applied to them, the protons all align their spins to $B_0$, resulting in a net magnetization. The spins will also begin resonating at the Larmor frequency, defined as $f = \frac{\gamma}{2\pi} B$. 

$\gamma$ is the gyromagnetic ratio and is specific to atoms. For a proton, $\frac{\gamma}{2\pi} = 42.58 Mhz/Tesla$. 

MRI machines also have Radiofrequency (RF) coils for the purposes of excitation and signal reception. To generate a signal, a radiofrequency pulse $B_1$ corresponding to the resonant frequency is applied in the transverse plane to $B_0$. (add explanation and images). These rotating magnetization moments induce an electromotive force (EMF) in the RF receiver coil (which may or may not be the same coil used for excitation), and the generated signal is called a free induction decay. 

Generalizing to 1D, the magnetization that is generated at position x can be expressed as $m(x)$, and because the magnetization vectors are rotating at frequency $\omega$, the collected signal generated from position x is $m(x)e^{i\omega(x)t}$ <span style="color:blue"> [1] </span>. In our current setup, there is only a single frequency $w_0$, so this becomes $m(x)e^{i\omega_0 t}$. The total collected signal across the entire image is $s(t) = \int_x m(x)e^{i \omega_0 t} dx$. We've now collected a signal from the entire sum of our image, since every spatial position has the same resonance frequency $\omega_0 = \gamma B_0$. This isn't quite useful - the collected signal is just a complex exponential with a constant magnitude.

The key idea here is to add a Gradient field, such that the resonance frequency is spatially varying instead of constant. The gradient field is an extra magnetic field $B_1$ in the same direction as $B_0$, except instead of being uniform, it varies with position in the transverse plane. For example, we could have a gradient that varies linearly in the x direction, such that $B_1(x) = Gx$ where G is a constant. Then, each spatial position will be mapped to a different temporal frequency, such that $\omega(x) = \omega_0 + \gamma Gx$! 

Then, the signal generated from position x is $m(x)e^{i (\omega_0 + \gamma Gx) t}$ and the total collected signal is $s(t) = \int_x m(x)e^{i (\omega_0 + \gamma Gx) t} dx = e^{i \omega_0 t} \int_x m(x) e^{i \gamma Gx t} dx$. We can demodulate the signal to remove the $e^{i \omega_0 t}$ term to get the baseband signal $\hat{s}(t) = \int_x m(x) e^{i \gamma Gx t} dx = F(\gamma G t)$ where $F(\omega)$ is the Fourier transform of $m(x)$, the magnetization. Thus, we can collect data on the Fourier domain of magnetization signal as a function of spatial position, and the inverse Fourier transform allows us to get an image of the magnetization, or proton density of an image!

Different tissues in the body have different hydrogen atom (proton) densities, and they emit signals with different strengths and patterns. By analyzing these signals, MRI can produce highly detailed images of the body's organs, soft tissues, bones, and other structures. 

In addition to producing static images, MRI can also be used to produce dynamic images of the body in motion, such as blood flow or joint movement. This is achieved by using different MRI techniques that capture the movement and changes in the signals emitted by the hydrogen atoms in the body. 


<span style="color:blue"> [1] </span>The actual signal equation is a lot more complicated - but we will use this generalized signal equation for the purposes of developing a basic understanding.

# Outline
As a recap, the basic idea behind MRI is that we use properties of magnetism to collect a signal corresponding to the frequency domain of an image, represented as  $\hat{s}(t) = \int_x m(x) e^{i \gamma Gx t} dx = F(\gamma G t)$,  where $F(\omega)$ is the Fourier transform of $m(x)$, the proton density of an object. The signal has to be processed digitally, so it is a discrete sampled version of the true signal. 

To build an understanding of the relationship of the Fourier transform and MRI, we will explore some key ideas:

1. Many signals are aperiodic, which means that they have an infinite number of Fourier components (a CTFT that extends infinitely long). However, $\hat{s}(t)$ can only be sampled to a finite range (we can't sit there and scan for an infinite amount of time), which may cause reconstruction artifacts.
2. We can only collect discrete samples of the frequency domain due to physical constraints of hardware. Images and signals in real life are continuous, but we can only collect discrete samples, which may cause aliasing.
3. MR images are 2D or 3D - so we will learn about the multidimensional Fourier transform.
4. Signals in EE120 are represented as a function of time, but images are instead a function of spatial position. Therefore, we will have to work with spatial frequency (kspace) instead of temporal frequency.
5. We will apply all these ideas to real scan data!


# Helper Functions
Please use the following helper functions for questions 1 and 2. Use stemplot for plotting a 1d discrete signal, and fftc/ifftc for performing Fourier transforms.

In imaging, we commonly deal with signals where the origin is at the center of the signal instead of at the 0th element of the array. However, generic FFT expects the origin to be located at the first element, so we have to do some shifting to account for this. We can use np.fft.fftshift and np.fft.ifftshift to shift a centered signal into a format that numpy likes (note that ifftshift does not correspond to the ifft - rather it is just the inverse operator of the fftshift, and they can theoretically be done in either order).

We also deal with complex valued signals, but for convenience we will only plot the real portion of 1d signals for easy of certain visualizations (such as sinc apodization). 

In [ ]:
def stemplot(signal, title = ''):
    plt.figure()
    plt.axis('off')
    plt.stem(np.real(signal))
    plt.title(title)
def fftc(f):
    return np.fft.fftshift(np.fft.fft(np.fft.ifftshift(f)))

def ifftc(F):
    return np.fft.fftshift(np.fft.ifft(np.fft.ifftshift(F)))


# 1. Finite support sampling

Let's try to build an understanding of what the effects are of sampling a finite range of the frequency domain. We will model the effect of finite sampling as an impulse response by observing the effects of finite extent sampling on an impulse.

Defined for you is an impulse (for now, we are modelling this as a dirac delta in continuous time. You can assume that the number of samples is high enough to accurately describe the continuous time signal - we will explore the effects of discrete time sampling in the next question).

Note that we will use k-space (abbreviated as ksp) to refer to the frequency domain. This will be explored more in a later question.

In [ ]:
impulse = np.zeros((63))
impulse[31] = 1
stemplot(impulse, "Impulse")

What does Poisson's identity (or just performing the CTFT) say about the transform of an impulse? Run the cell below to confirm your answer.

In [ ]:
ksp_impulse = fftc(impulse)
stemplot(ksp_impulse, "Fourier Transform of Impulse Response")


# 1a: Low Pass Filter
Now let's observe the effects of only sampling the center of the frequency domain and then reconstructing our signal. 
Fill in the function below to generate a low pass filter signal of the same length of freqs, where the innermost 1/3 of the filter signal is 1, and the rest of filter signal is 0.

In [ ]:
def low_pass_filter(freqs):
    '''
    Returns a low pass filter/rect signal of the same length as freqs. 
        The filter is 1 for the innermost 1/3 of its length (starting at length//3) and 0 elsewhere.
    '''
    rect = None
    
    ...
    return rect

In [ ]:
grader.check("q1a")

In [ ]:
LPF = low_pass_filter(ksp_impulse)
stemplot(LPF , "Low pass filter in frequency domain")
stemplot(ifftc(LPF) , "Low pass filter in time domain")

This is consistent with the Fourier transform pair of a rect function in one domain being equivalent to a sinc in the other domain.

Let's see what happens when we apply the filter to our impuslse signal and then reconstruct.

In [ ]:
stemplot(ifftc(LPF*ksp_impulse), "Inverse transform of Low Pass Filter Applied to Impulse")

Notice that this is exactly a sinc - the same sinc we saw earlier.

Recall the convolution theorem - convolution in one domain corresponds to multiplication in the other domain. When we multiplied the frequency domain signal by the rect function, this is equivalent to convolving the time domain signal by the inverse Fourier transform of the rect function (which is a sinc). Therefore, multiplying the rect in the frequency domain by the with the Fourier transform of the impulse is equivalent to convolving a sinc with an impulse in the time domain, which returns the exact same sinc (by the identity property of the impulse).

Fill in the function below to perform a limited bandwidth sampling of a signal. Your task is to convert the signal from the time domain to the frequency domain, zero out the first and last 3rds of the frequency domain, and then convert it back into the time domain. The low pass filter function may be useful!

# 1b: Finite Sampling


In [ ]:
def finite_sample(signal):
    '''
    Performs a limited bandwidth sampling of the signal by zeroing out 
        the first and last 3rds of the frequency domain (FFT) of the signal.
    '''
    ...

In [ ]:
grader.check("q1b")

In [ ]:
sparse_signal = np.zeros((63))
sparse_signal[16] = 1
sparse_signal[48] = 1
stemplot(sparse_signal, "Two Impulses")
sparse_lowres = finite_sample(sparse_signal)
stemplot(sparse_lowres, "Apodization of Two Impulses")

# Q1c: Analysis

Q: Treating finite sampling as an LTI system, what is the impulse response of the system? What is the output in terms of an input to the system?

Q: Looking at finite sampling from a Fourier perspective, what is the effect of apodization? 

<!-- BEGIN QUESTION -->

<span style="color:blue">**A:** (TODO)</span>

<!-- END QUESTION -->

# 2a: Discrete Sampling

Now let's look at the effects of sampling discrete timesteps of a continuous signal.

Fill in the function below to perform a downsampled reconstruction. Convert the signal into the frequency domain, downsample its frequency domain components, and then reconstruct the downsampled signal.

In [ ]:
def discrete_sample(signal, k):
    '''
    Samples every kth value of the frequency domain (i.e. zeros out the other values) and reconstructs the signal. 
    When sampling, make sure you start at the 0th element.
    '''
    
    length = len(signal)
    ksp = fftc(signal)
    ksp_sampled = None
    ...
    return ifftc(ksp_sampled)



In [ ]:
grader.check("q2a")

In [ ]:
impulse = np.zeros((256))
impulse[10] = 1
stemplot(impulse)
stemplot(discrete_sample(impulse, 2), "Impulse with aliasing")

A copy occured at exact half of the signal's length away!

Let's see what happens to a generic signal. (Note: due to discrete and finite sampling, odd effects may occur if your signal length or k is not a power of 2 - but don't worry about those.)

In [ ]:
signal = np.zeros((128))
signal[0] = 1
signal[1] = 2
signal[3] = 3
stemplot(signal, "DT Signal")
stemplot(discrete_sample(signal, 4), "Signal with aliasing")

# 2b: Analysis

This phenomenon is called **aliasing** and occurs when we don't sample at a high enough rate.
Let's consider why this happens. The DFT takes a signal with a certain Field of View ($FOV$) and pretends it periodically replicates outside of that $FOV$, and therefore the signal can be broken down into complex exponentials corresponding to $w = \frac{2\pi*k}{FOV}$, aka complex exponentials every $\Delta w = \frac{2\pi}{FOV}$. 

When we zero out every other collected frequency sample,
1. What is the equivalent new $\hat{\Delta w}$, 
2. What is the corresponding $\hat{FOV}$?
3. What happens to the corresponding image in the original $FOV$?
4. Generally, if we sample every kth value of the frequency domain, how many copies of the original image will appear in the time domain?


<!-- BEGIN QUESTION -->

<span style="color:blue">**A:** (TODO)</span>

<!-- END QUESTION -->

# 2c: Finite and Discrete Sampling

Putting it all together - let's take a look at what happens when we apply finite and discrete sampling to a signal. (There is nothing you have to answer for this question)

In [ ]:
box = np.zeros((128))
box[34:54] = 1
stemplot(box, "DT Box")
sampled_box = finite_sample(discrete_sample(box, 2))
stemplot(sampled_box, "Sampled Box")


A few things should stand out - firstly, there are now two copies of the signal, spaced apart by 1/2 FOV. This is because of the discrete sampling - when we sampled half as often, we got 2 copies of the signal; aliasing occured. This can be prevented as long as our samping rate is high enough.

Notice that each copy of the box isn't exactly a perfect rectangle. Instead of a top, flat peak, there are some sinc-like curvatures - this effect is known as blurring. Also, notice that outside the width of the box, there are some sinc-like oscillations - this is known as Gibbs ringing. These latter two effects can be summarized as sinc apodization caused by limited bandwidth sampling, and cannot be removed (but can be minimized by collected a high extent of the frequency domain).

If you are done with this section, please run the following code block to preserve memory.

In [ ]:
plt.close('all')

# 3. 2D FT

Most images that are dealt with in practice are 2D (the semantics of the dimensions will be discussed in the next question), so we must update our toolbox accordingly.

The 2D Fourier transform (for continuous time) is defined as 

\begin{equation}
F(k_x, k_y) = \int_{-\infty}^{\infty}\int_{-\infty}^{\infty}f(x,y)e^{-i2\pi(k_xx + k_yy)}dxdy
\end{equation}




# 3a) 2D DFT

The multidimensional DFT is defined as such:

$$ X_{\textbf{k}} = \sum_{\textbf{n} = 0}^{\textbf{N-1}} e^{-i 2\pi \textbf{k}\cdot \textbf{(n/N)}}x_{\textbf{n}}$$

Note that $\textbf{k}, \textbf{n}, \textbf{N}$ are all vectors (and $\textbf{(n/N)}$ represents elementwise division). 

Generalizing to 2D and de-vectorizing, the formula becomes:

$$ X[k, l] = \sum_{m=0}^{M-1} \sum _{n=0}^{N-1} x[m,n]  e^{-i 2\pi (\frac{mk}{M} + \frac{nl}{N})}$$

Implement the 2D DFT function below. You may not use np.fft.fft2.

In [ ]:
def dft2d(x):
    M, N = x.shape
    X = np.zeros(x.shape, dtype=np.complex128) # output array - have to declare as complex so it's not cast to real
    
    ...
    return X

Some tests:

In [ ]:
grader.check("q3a")

# 3b) 2D FFT
WAIT - don't close the notebook just yet, there's no divide and conquer recursion involved in this question! 

One nice property about the 2D DFT is that it is equivalent to first taking the DFT along each column along (for each column), and then taking the DFT along each row (for each row). The order doesn't actually matter, so you could also do rows first and then columns. Therefore, a one-dimensional FFT algorithm can be used to easily implement a multidimensional FFT algorithm, iteratively along each dimension.

Implment the 2D FFT function below, using np.fft.fft as a helper. You may not use np.fft.fft2. 

Hint: the parameters of np.fft.fft are (a, n=None, axis=-1, norm=None). This function computes the FFT along a single dimension, and the dimension is chosen by the axis parameter. 

In [ ]:
def fft2d(x):
    
    ...

In [ ]:
grader.check("q3b")

# 3c) 2D iFFT

The multidimensional iFFT works in the exact same sense that the FFT does - you can just compute it along each axis independently. Fill in the function below for a 2D iFFT.

In [ ]:
def ifft2d(X):
    
    ...

In [ ]:
grader.check("q3c")

# 3d) FFT shift
Just like with 1d signals, the imaging convention of the origin is at the center of the image, instead of at the (0,0)th element (which would be the top left). Fill in the fft2dc (centered) and ifft2dc functions for use on a centered image. 

Hint: fftc and ifftc for 1d signals were provided to you at the beginning of the lab - the idea will be very, very similar.

In [ ]:
def fft2dc(f):
    ...

def ifft2dc(F):
    ...

In [ ]:
grader.check("q3d")

For the rest of this lab, we'll be dealing with 2D images, so please use your fft2dc and ifft2dc functions, as well as the provided function below for plotting 2D images.

In [ ]:
def implot(image):
    plt.figure()
    plt.axis('off')
    plt.imshow(np.abs(image).T, cmap = 'gray', origin = 'lower')

# 4. K-Space 

In imaging, we deal with **spatial frequency** instead of temporal frequency, and we also commonly use 2D images, which have frequencies along both the x axis and y axis, denoted as $k_x, k_y$ respectively (note that we aren't using angular frequency $\omega$, but rather frequency corresponding to $\frac{cycles}{cm}$, similar to Hertz for temporal frequency). 

To calculate a spatial frequency, it is similar to calculating temporal frequency. Looking at the image below, first we notice that it is constant along the y-axis, so the frequency along the y-axis is 0. Along the x axis, we notice that the signal repeats every 10 cm, which corresponds to $10 \frac{cm}{cycle}$, which corresponds to $k_x = 0.1 \frac{cycle}{cm}$. 

In [ ]:
nx = 100
ny = 100
x = np.linspace(-50,50,nx)
y = np.linspace(-50,50,ny)
X,Y = np.meshgrid(x,y)
kx = 0.1
Z = np.cos(2*np.pi*kx*X + 0*Y)
plt.figure()
plt.xlabel("cm in x")
plt.ylabel("cm in y")
plt.imshow(Z, cmap='gray', extent=[-50, 50, -50, 50], origin="lower")

Notice that the image above looks like a sinsoid (specifically, a cosine) along the x axis, which means that its Fourier transform should be an impulse at the corresponding positive and negative frequency.

In [ ]:

FZ = fft2dc(Z)
plt.figure()
plt.xlabel("cm^-1 in x")
plt.ylabel("cm^-1 in y")
plt.imshow(np.abs(FZ), cmap='gray', extent=[-0.5, 0.5, -0.5, 0.5])


It's a bit trickier when the frequency isn't axis aligned, but we can do a similar process here. Look at just the x axis and see how far it takes for a single cycle, and do the same along the y axis to find the spatial frequencies, which in this case are 0.1 and 0.05.

In [ ]:
nx = 100
ny = 100
x = np.linspace(-50,50,nx)
y = np.linspace(-50,50,ny)
X,Y = np.meshgrid(x,y)
fx = 0.1
fy = 0.05
Z = np.cos(2*np.pi*fx*X + 2*np.pi*fy*Y)
plt.figure()
plt.xlabel("cm in x")
plt.ylabel("cm in y")
plt.imshow(Z, cmap='gray', extent=[-50, 50, -50, 50], origin="lower")

In [ ]:

FZ = fft2dc(Z)
fig, ax = plt.subplots(figsize=(6,6))
plt.xlabel("cm^-1 in x")
plt.ylabel("cm^-1 in y")
ax.imshow(np.abs(FZ), cmap='gray', extent=[-0.5, 0.5, -0.5, 0.5], origin="lower")

# 4a: Spatial Mystery
Now it's your turn - what are the frequencies fx and fy of the following 2D cosine function? Fill in the code block below and make sure your plotted image is the same as the mystery.

[<img src="images/sinusoid.PNG" width="500"/>](images/sinusoid.PNG)


In [ ]:
nx = 100
ny = 100
x = np.linspace(-50,50,nx)
y = np.linspace(-50,50,ny)
X,Y = np.meshgrid(x,y)

fxQ4a = None
fyQ4b = None

...
Z = np.cos(2*np.pi*fxQ4a*X + 2*np.pi*fyQ4a*Y)
plt.figure()
plt.xlabel("cm in x")
plt.ylabel("cm in y")
plt.imshow(Z, cmap='gray', extent=[-50, 50, -50, 50], origin="lower")


# 4b: Frequency Mystery
Now, let's solve a mystery in the frequency domain! Given the following kspace spectrum, solve for the function that would produce this frequency spectrum. 
[<img src="images/box.PNG" width="500"/>](images/box.PNG)


Hint: For 2D Fourier transforms, $$f(x)f(y) \iff F(k_x)F(k_y)$$ Along one direction (x or y) of the spectrum, what does the image look like? What is the corresponding inverse transform of that function? You should be able to use a predefined numpy mathematical function.

**Note: Do not apply any scaling factor to the function.**

In [ ]:
nx = 100
ny = 100
x = np.linspace(-50,50,nx)
y = np.linspace(-50,50,ny)
X,Y = np.meshgrid(x,y)
fx = 0.5
fy = 0.2
funcMystery = None
...
Z = funcMystery(fx*X) * funcMystery(fy*Y)
plt.figure()
plt.xlabel("cm in x")
plt.ylabel("cm in y")
plt.imshow(Z, cmap='gray', extent=[-50, 50, -50, 50], origin="lower")

Run the code block below to make sure the frequency spectrum looks correct.

In [ ]:

FZ = fft2dc(Z)
fig, ax = plt.subplots(figsize=(6,6))
plt.xlabel("cm^-1 in x")
plt.ylabel("cm^-1 in y")
ax.imshow(np.abs(FZ), cmap='gray', extent=[-0.5, 0.5, -0.5, 0.5], origin="lower")

# 5. Brains and Plotting

We finally get to the exciting part, brains!!

Let's try applying our knowledge of sampling to some real brain data from an incredibly smart, talented, funny, and handsome brain scan volunteer. 

The k-space (frequency domain) data has been provided to you in a numpy array. 

# 5a) Basic Reconstruction
Reconstruct and plot the image of the brain. The code is given to you for this question (you do not need to write anything for 5a). 

In [ ]:
ksp = np.load('ksp.npy')
print(ksp.shape)

In [ ]:
# BEGIN SOLUTION
im = ifft2dc(ksp)
implot(im)
# END SOLUTION 


# 5b) Bandlimited Sampling
Now, try simulating a bandlimited sample of the reconstruction where you only collected 1/3 of the frequency domain in both directions. Plot the resulting image.

<!-- BEGIN QUESTION -->



In [ ]:
ksp_crop = ksp.copy()

...

<!-- END QUESTION -->

# 5c) Sub-Nyquist Sampling
Now try simulating a reconstruction where your sampling rate **in the x direction** was half of what is was before (i.e. you only acquired every other sample **in the x direction**). Plot the resulting image.

<!-- BEGIN QUESTION -->



In [ ]:
ksp_samp = ksp.copy()

...

<!-- END QUESTION -->

# 5d) Mystery
Uh oh, a bug happened when you were collecting data and you got the following image:
[<img src="images/mystery.PNG" width="350"/>](images/mystery.PNG)

What could've happened to your data to produce an image like this? Fill in the missing line in the code block below to reproduce this image. 

Hint: The whole image was shifted by half of the FOV. What frequency does that correspond to, and what simple geometric sequence would produce it?

In [ ]:
ksp_mystery = ksp.copy()

mystery = None
...
ksp_mystery[::2] *= mystery

im_mystery = ifft2dc(ksp_mystery)
implot(im_mystery)

## Conclusion

In this lab, we went over the general idea behind MRI and its relationship to the Fourier transform. We explored the effects of finite and discrete sampling, learned about the multidimensional Fourier transform and spatial frequencies, and solved a k-space mystery! 

If you're interested in MRI, check out EE225E, Berkeley's very own class on the Principles of Magnetic Resonance Imaging.

## References
 

* [1] [Nishimura, Dwight. "Principles of Magnetic Resonance Imaging"](https://www.lulu.com/shop/dwight-nishimura/principles-of-magnetic-resonance-imaging/paperback/product-1nqdq4j2.html?page=1&pageSize=4)
* [2] [EE 225E Course Website and Slides](https://sites.google.com/berkeley.edu/ee225ebioe265)
* [3] [Wikipedia on DFT](https://en.wikipedia.org/wiki/Discrete_Fourier_transform#Multidimensional_DFT
)
* [4] [Wikipedia on FFT](https://en.wikipedia.org/wiki/Fast_Fourier_transform#Multidimensional_FFTs
)
* [5] [ECE533 at University of Arizona Course Slides](https://uweb.engr.arizona.edu/~dial/ece533/notes9.pdf
)
* [6] [Barteezy's blog on image processing ](https://barteezy.wordpress.com/2015/09/30/activity-6-properties-and-applications-of-the-2d-fourier-transform/)




## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit.

Please upload the zip file produced by the result of this command to Gradescope.

In [ ]:
grader.export(force_save=True, run_tests=True)